
# Libraries and Tools used

In [2]:
"""Our graphical elements are going to be done with matplotlib, inline graphs"""
%matplotlib inline
import matplotlib.pyplot as plt

"""CSV: utility to work with the most common format for datastorage : Comma Separated Values"""
import csv

"""JSON: utility to work with JavaScript Object Notation, the original information format storage we """
import json
from collections import namedtuple

"""Pandas: python's most used library to work with datasets"""
import pandas as pd 

""" Type detection for our dataframe """
from pandas.api.types import (
    is_categorical_dtype,
    is_datetime64_dtype,
    is_object_dtype,
    is_numeric_dtype,
    is_string_dtype
)

"""Numpy: python's most used library to work with large amounts of numbers"""
import numpy as np
"""OS: utility to work with paths and file openings independently of operating system"""
import os
"""Glob: used to look for file extensions inside given folders"""
import glob 

"""Candidates for ML predictive model implementations"""
import tensorflow as tf
import sklearn
import catboost

/Users/brick/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# The Storage module

### Methods 

#### __init__
#### print_info
#### from_json
#### to_json
#### update_stats
#### load_from_dataframe


In [8]:
class StoreDomain():
    
    def __init__(self, attribute_list, path=os.getcwd(), domain_name=None):
        """ Handy initialization """
        self.load(attribute_list, path, domain_name)
        
    def load(self, attribute_list, path, domain_name):
        """ Given a dataset searches for a domain in the specified folder.
            If found, loads it into the class itself. If not found loads an empty template.
        """
        for fnm in glob.glob(os.path.join(path,"*.json")):
            with open(fnm) as f:
                data = json.load(f)
            if data["attribute_list"] == attribute_list:
                self.modify(data)
                self.fnm = fnm
                break;
                        
        if not hasattr(self,"fnm"):
            print("No matching knowledge found for your domain, setting an empty one")
            self.attribute_list = attribute_list
            self.domain_name = domain_name or attribute_list
            self.knowledge = {
                "dataset_stats" : {},
                "column_stats": {}
            }
            self.fnm = glob.glob(os.path.join( path, self.domain_name+".json") )
                    
    def save(self):
        """ Saves itself as an object in .json format
        """
        print("" "", self.fnm)
        with open(self.fnm,'w') as f:
            json.dump(self.__dict__, f)
            
    def modify(self,data):
        """ Modifies the class from a physical representation (dict) of it
        """   
        self.domain_name = data["domain_name"] if "domain_name" in data else self.domain_name
        self.attribute_list = data["attribute_list"] if "attribute_list" in data else self.attribute_list
        self.knowledge = data["knowledge"] if "knowledge" in data else self.knowledge
        self.fnm = data["fnm"] if "fnm" in data else self.fnm

    def print_info(self):
        """ Util to print attributes
        """
        print(self.__dict__)


In [11]:
## StoreDomain tests

df = pd.read_csv("nonsense.csv")
del df['Unnamed: 0']

df.name = "nonzenze"
std = StoreDomain(list(df.columns),".",  df.name)

std.save()
std.print_info()
#dom = sd.domain_from_dataset(ds,paths)

 ./nonzenze.json
{'domain_name': 'nonzenze', 'attribute_list': ['a', 'b'], 'knowledge': {}, 'fnm': './nonzenze.json'}


# The JSON
&nbsp;

 This object is used to represent the raw information stored.

 It contains statistics and properties from both statistics and datasets, different to different types, as well as how they were measured (by saving the function to be called on a pandas dataset in the case of a dataset stat, or on the values themselves in the case of a function stat).

 Saving how they're measured is important to measure new datasets and to be able to compare metrics effectively.

 Both the column specific stats and the dataset stats are subjective to change, and the knowledgedomain object can be modified and adapted to fit new parameters easily.

 This structure is generated for a single dataset and then combined with the domain one to take account of the new one.



In [7]:
KNOWLEDGE_STRUCTURE = {
    "dataset_stats": {
        "individuals": 0,
        "max_rows": {
            "value": 1400,
            "metric" : "count_rows_null",
        }, # alert if new rows are <<<, as different results are skewed
        "avg_rows": {
            "value": 1400,
            "metric" : "count_rows_null"
        },
        "full_rows": {
            "value": 0.97
        }
    },
    "column_stats": {
        "a": {
            "type": "numeric",
            "stats": {
                "median": 12.6,
                "std_dev": 1.3,
                "max": 25.5,
                "min": 2.3,
                "NaNs": 0.02 # as % of dataset
            }     
        },
        "b": {
            "type": "categorical",
            "stats": {
                "most_frequent": "cat",
                "values": {
                    "cat": 0.2,
                    "dog": 0.6
                },
                "nan": 0.1
            }
            
        }

    }
}


# The analysis module
## Metrics

In [ ]:
""" The purpose of these functions is to provide a way to measure the properties of a given dataset or knowledge domain.
    We can categorize them as follows:
    First the "measurement" metrics, used to get the information of a single dataset or domain.
    
    - Dataset Metrics : they concern the dataset as a whole, like number of rows with missing values.
        dm :: (ds) --> num
        
    - Single Column Metrics : they concern a certain column, and are based on the type of the column.
      For numerical columns we will have things like median, averages, deviations, distributions...
      For categorical columns we'll work with frequencies and things of the sort.
        scm :: (col) --> num
        
    - Multiple Column Metrics : we will be looking for correlations and things of that sort.
       scm :: (col,col) --> num 
       Time based metrics will be defined from this construct.
    
    We will also have "comparison" metrics, used to compare datasets against their domains.
    These metrics will compare the output of two measurement metrics, both will have to
    spawn from the same function.
        comp_m :: (metric) --> num
    
    Note that these metrics are not to provide "meaning" or any human-readable input, nor to be
    inherently comparable between each other outside of a framework of understanding of the domain
    (metric importance).
    
    A mean to convert these machine cold metrics into human understanding will be provided in further 
    modules. For now, we're not taking humans into account.
"""

""" Dataset Metrics """

def count_rows_null(df):
    return df.isnull().shape(0)

""" Columns """

""" For consistency, every column is accepted as a pandas Series """
""" Numeric, accepted as pandas Series """

def median(col):
    return np.median(col.values)

def average(col):
    return np.average(col.values)



""" All types """

def count_not_null(col):
    return np.count_nonzero(~np.isnan(data))

def count_null(col):
    return col.size - count_not_null(col)



""" Categorical """

def count_freqs(col):
    return 1.2
def count_nans(col):
    return 0.9
""" Datetime """



## Utility functions

In [8]:

IS_DTYPE = {
    "number": is_numeric_dtype,
    "category": is_categorical_dtype,
    "datetime": is_datetime64_dtype,
}

DEFAULT_METRICS = { #(type,comparison_metric)
    "numeric" : 3
    "categorical": 2
    "dataset" : {
        "count_rows":{
            "metric": 2
        },
        "rows_with_nulls":{
            "metric": 3
        }
    }
}

COMPARISON_METRICS = { #(metric(name of a function) , comparison_metric)
    
}

In [4]:

class DatasetAnalyzer():
    
    def __init__(self, df):
        self.dataset = df            
        self.domain_knowledge = StoreDomain(df.name, list(df.columns))
        self.dataset_knowledge = self.get_stats(df)
        
    def get_stats(self, df):
        """ Populates the stats dictionary """
        stats["column_stats"] = self.get_column_stats(df)
        stats["dataset_stats"] = self.get_dataset_stats(df)
        #if k has been correctly created it is assumed to have a metric field
        return stats
    
    def get_column_stats(self,df):
        """ Column stats """

        stats = {}
        col_types = self.get_column_types(df)
        
        for c in self.get_column_types(df):
            if c in self.domain_knowledge["column_stats"]:
                func = globals()[self.knowledge["column_stats"][c]["metric"]]
            else:
                func = DEFAULT_METRICS[col_types[c]]
                
            stats[c] = {
                    "metric" : func,
                    "value" : func(df[c].values)
            }
        return stats
    
    def get_dataset_stats(self,df):
        """ Dataset stats """
        stats = {}
        dataset_metrics = self.fill_d1_with_d2(self.domain_knowledge["dataset_stats"])
        for m in dataset_metrics:
            stats[m] = {
                "metric": globals()[dataset_metrics[m]["metric"]],
                "value": globals()[dataset_metrics[m]["metric"]](df)
            }            
        return stats
    
    def get_column_types(self, df):
        """ Get column types """
        col_types = {}
        for c in df.columns:
            col_types[c] = "numerical" if IS_DTYPE['number'](col.dtype) else "categorical"
            
        return col_types
    
    def fill_d1_with_d2(self, d1, d2):
        """Add extra stats from d2 (default) to d1 (our dict)"""
        for key in d2:
            if (key in d1 and isinstance(d1[key], dict) and isinstance(d2[key], dict)):
                self.deep_merge_dicts(d1[key], d2[key])
            else:
                d1[key] = d2[key]

    def run_analysis(self): #generates report, saves it in self.report AND returns it.
        """ This method will return a JSON containing the comparison between the dataset knowledge
            and the domain knowledge
        """
        pass
        # load self dataset knowledge
        # load self domain knowledge
        # for every stat in stats just compare them using function provided by dictionary(metric,comparison_metric)
                    
    def update_stats(self): #mixes new data with the old, for which we have to have a param with instances
        # get every stat in dataset knowledge
        # get every stat in domain knowledge
        # combine them with weighted means (we need a # of historic datasets param )
        pass

# CBR Database

In [28]:
class StoreCBR(StoreDomain):
    """ StoreDomain but for human knowledge. Extends StoreDomain to add profile functionality
        and CBR-based behaviour.
        Each storecbr is tied to a profile and it can only modify it
    """
    def __init__(self, attribute_list, profile, path, domain_name=None):
        """ Handy initialization """
        self.load(attribute_list, profile, path, domain_name)
        
    def load(self, attribute_list, profile, path, domain_name):
        """ Given a dataset searches for a domain in the specified folder(s) and loads
        """
        for fnm in glob.glob(os.path.join(path,"*.json")):
            with open(fnm) as f:
                data = json.load(f)
            if data["attribute_list"] == attribute_list:
                self.domain_name = data["domain_name"] if "domain_name" in data else self.domain_name
                self.attribute_list = data["attribute_list"] if "attribute_list" in data else self.attribute_list
                self.knowledge = data["knowledge"] if "knowledge" in data else self.knowledge
                self.profile = profile
                self.fnm = fnm
                break;
                        
        if not hasattr(self,"fnm"):
            self.set_defaults(domain_name ,attribute_list, path, profile)
            
    def set_defaults(self, domain_name ,attribute_list, path, profile):
        """
        """
        self.domain_name = domain_name or attribute_list
        self.attribute_list = attribute_list
        self.profile = profile

        self.fnm = glob.glob(os.path.join( path, self.domain_name+".json") )
        self.knowledge = {}
        self.knowledge["profiles"][profile] = {
            "profile_knowledge" : {}
        }
        
    def modify(self, new_info):
        self.knowledge["profiles"][profile] = new_info
        
    def run_tournament(self): pass

# Reporter
### Human-machine hybrid


In [21]:
""" 
    Another kind of information is stored about the domains. This is the 
    information concerning the **human** side of things, that is, **how** 
    to interpret these stats and turn them into something that humans with different levels of 
    familiarity can understand.
    To do this, we provide use another storage class that will contain human-relevant data that
    will modify the objective comparison delivered by the analysis module.
"""

class Reporter():
    
    def __init__(self, machine_info, profile, human_info=None):
        self.profile = profile
        self.machine_info = machine_info
        self.human_info = human_info if human_info else StoreCBR(columns=analysis_result["attribute_list"],profile=profile)

    def generate(self):
        for k in self.machine_info:
            
        # compare the result analysis with our knowledge of the domain, then select the relevant stuff, store json
        
        #in self.report_json
        
        return 0
    def modify(self,new_info):
        self.human_info.modify(new_info)
    def save_human_info(self,new_info):
        self.human_info.save(new_info)

# Frontend module
Logic-free, just turns an information json to a user-readable report with images, which then prints out for the user to see.

In [ ]:
class TextTemplates:
    def __init__(self):
    def red_json(self,):
PRETTY_JSON = {
    "graphs": {
        "some_col":{
            "type" : "bars",
            "cat_colors" : {
                "cat" : "red",
                "dog" : "blue",
            },
            "bg_color" : "white"
        }
    },
    "text" : {
        "some_col_avg" : {
            "reason" : "high",
            "number_ds" : 1.3,
            "number_domain": 0.2
        }
    }
}

In [10]:
class FrontEnd:
    def __init__(self, config):
        """Self explaining, associate graph with """
        self.graphs_config = config["graphs"]
        self.text_config = config["text"]
    def generate_graphs(self):
        for config in self.graphs_config:
            print(config)
    def generate_text(self):
        for config in self.text_config:
            print()
        #text things
        #graphical things
        return 0

In [9]:
EXEC_PARAMS = {
    "domain_name" : "animals",
    "attribute_list": [] ,
    "knowledge" : {}
}
if __name__ == "__main__":
    """First we read our data"""
    df = pd.read_csv("nonsense.csv")
    dsa = DatasetAnalyzer(df)
    #load dataset into analyzer
    #get report generator
    #use the froentend module to present it to the user
    #if the user gives feedback, frontend saves it as important in the store (?) information
    print("henlo")

NameError: name 'DatasetAnalyzer' is not defined